In [1]:
from data_encode import encode
from data_encode import to_byte

In [ ]:
encoded_message = encode(string_input)

In [3]:
alpha = 0x02 #Alpha is an an element which can generate all elements of the field after being raiesd to some power
primitive = 0x11D #Primitive is irreductible polynomial that defines the Gallois Field.

log = [0]*256
antilog = [0]*512 #Since the operations are closed so there must be wraparound i.e. alpha^255 = alpha^0 = 1 so when there is antilog(log[a] + log[b]), antilog(log[a] + log[b] % 255) is not required since the values beyond 255 are kept the same as the initial values.

x = 1

for i in range(255):
    antilog[i] = x
    log[x] = i

    x = x<<1 #Left Shift by 1 i.e. multiply by 2 i.e. alpha; microprocessor nostalgia
    
    if x & 0x100: #If there is 9th bit then it is beyond GF(256)
        x = x^primitive #EXOR is same as addition/substraction, later in this case, in GF(2)

for i in range(255, 512):
    antilog[i] = antilog[i-255]

In [5]:
def gf_addition(a, b):
    return a^b

def gf_multiplication(a, b):
    if a==0 or b==0:
        return 0
    return antilog[log[a] + log[b]]

def gf_division(a, b):
    if a==0:
        return 0
    if b==0:
        raise ZeroDivisionError()
    return antilog[log[a] - log[b]]

def poly_mul(a, b):
    result = [0]*(len(a)+len(b)-1)
    for i in range(len(a)):
        for j in range(len(b)):
            result[i+j] ^= gf_multiplication(a[i], b[j])
    return result

def generate_generator(num_ec):
    g = [1]
    for i in range(num_ec):
        g = poly_mul(g, [1, antilog[i]])  # (x-a^i) is written as [1, antilog[i] i.e. a^i] and +/- are same in GF(256)
    return g

def poly_div(dividend, divisor): #We perform polynomial long division after which the remainder will be the error correction codeword
    divi = list(dividend)
    for i in range(len(dividend) - len(divisor)+1):
        coeff = divi[i]
        if coeff!=0:
            for j in range(len(divisor)):
                divi[i+j] ^= gf_multiplication(divisor[j], coeff) 
                '''This is in place division where the dividend is modified in each operation 
                    divisor*highest_coeff_of_dividend is substracted from dividend taking the first terms equal to the number of terms of divisor.
                    The first term is always cancelled as the generator polynomial is monic and other terms are modified accordingly.'''
    remainder = divi[-(len(divisor)-1):]  #The last remaining terms with degree, at most, one less than divisor
    return remainder

In [ ]:
def get_error_correction(encoded_message, ERROR_CORRECTION_BYTES):
    padded_message = encoded_message + [0 for i in range(ERROR_CORRECTION_BYTES)]
    redundant_bits = poly_div(padded_message, generate_generator(ERROR_CORRECTION_BYTES))
    return redundant_bits


def split_blocks(coded_data, split_specs):
    # coded_data has data in list form
    # split_specs is a list of tuple specifying the number of blocks and size of each block, of each group

    blocks = []
    index = 0
    for (num_blocks, block_size) in split_specs:
        for _ in range(num_blocks):
            block = coded_data[index: index + block_size]
            blocks.append(block)
            index += block_size
    return blocks

def join_blocks(data_blocks, redundancy_blocks):
    data_interleaved = []
    redundancy_interleaved = []

    max_block_size = max(len(b) for b in data_blocks)

    for i in range(max_block_size):
        for block in data_blocks:
            if (i<len(block)):
                data_interleaved.append(block[i])

    for i in range(len(redundancy_blocks[0])):
        for block in redundancy_blocks:
            redundancy_interleaved.append(block[i])

    final_code_word = data_interleaved + redundancy_interleaved
    return final_code_word

def add_error_correction(encoded_message, split_specs, error_correction_bytes_number):
    split_message = split_blocks(encoded_message, split_specs)

    redundancy = []
    for block in split_message:
        block_error_correction = get_error_correction(block, error_correction_bytes_number)
        redundancy.append([block_error_correction])

    final_code_word = join_blocks(split_message, redundancy)
    return final_code_word
